# The Critics Critique ML model 1 

## 0. import libraries

In [2]:
import pandas as pd
import numpy as np
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

## 1. import and slice dataset 

In [3]:
all_sites = pd.read_csv('../data/all_sites.csv')

In [4]:
len(all_sites)

15551

In [5]:
all_test = all_sites[0:100]

In [8]:
all_test.head(3)

,site,url_link,author,game,company,genre,platform,text,score
0,meristation,https://as.com/meristation/2008/12/22/analisis...,Javier de Pascual,Metal Gear Solid Mobile,Konami,"Acción, Tactical",NGE,"Por alguna extraña razón, cuando el jugón trad...",8.5
1,meristation,https://as.com/meristation/2012/12/19/analisis...,Pablo González,THE DENPA MEN: They Came By Wave,None,RPG,3DS,Nintendo 3DS ha apostado desde su primer día e...,7.2
2,meristation,https://as.com/meristation/2019/09/03/analisis...,David Arroyo,Telling Lies,Annapurna Interactive,Aventura,PC PS4 XBO NSW,Hablar de Telling Lies es un poco complicado. ...,8.0


## 2.2 Classifier with Transformers nlptown/bert-base-multilingual-uncased-sentiment

In [5]:
# This let us see prob for every batch 1-5

'''from adaptnlp import EasySequenceClassifier

classifier = EasySequenceClassifier()

sentences = classifier.tag_text(
    text=text_splitted,
    model_name_or_path="nlptown/bert-base-multilingual-uncased-sentiment",
    padding='longest',
    mini_batch_size=4
)

print("Tag Score Outputs:\n")
for sentence in sentences:
    print(f"Fragment {sentences.index(sentence)}: {sentence.labels}")'''

'from adaptnlp import EasySequenceClassifier\n\nclassifier = EasySequenceClassifier()\n\nsentences = classifier.tag_text(\n    text=text_splitted,\n    model_name_or_path="nlptown/bert-base-multilingual-uncased-sentiment",\n    padding=\'longest\',\n    mini_batch_size=4\n)\n\nprint("Tag Score Outputs:\n")\nfor sentence in sentences:\n    print(f"Fragment {sentences.index(sentence)}: {sentence.labels}")'

### Sentiment proccesing function Bert base Multilingual Uncased 

In [10]:
def sentiment_analysis_ind(df):
    global_stars = []
    
    # Select model
    nlp_model = 'nlptown/bert-base-multilingual-uncased-sentiment'
    tokenizer = AutoTokenizer.from_pretrained(nlp_model)
    model = AutoModelForSequenceClassification.from_pretrained(nlp_model)

    # Classifier
    classifier = pipeline(
        'sentiment-analysis', 
        model=model, 
        tokenizer=tokenizer
    )
    
    # Adapt review lenght to max tokens
    n = 1900
    review_splitted = [(review[i:i+n]) for i in range(0, len(review), n)]
    #print(len(review_splitted))
    
    # Classifying 
    for text in review_splitted:
            stars = (classifier(text))
            global_stars.append(stars)

    return global_stars
    

In [11]:
def sentiment_analysis_bert_base_multilingual_uncased(review):
    global_stars = []
    n = 2000
    
    # Select model
    nlp_model = 'nlptown/bert-base-multilingual-uncased-sentiment'
    tokenizer = AutoTokenizer.from_pretrained(nlp_model)
    model = AutoModelForSequenceClassification.from_pretrained(nlp_model)

    # Classifier
    classifier = pipeline(
        'sentiment-analysis', 
        model=model, 
        tokenizer=tokenizer
    )
    
    review_splitted = [(review[i:i+n]) for i in range(0, len(review), n)]    
    
    for text in review_splitted:
        stars = (classifier(text))
        global_stars.append(stars)

    return global_stars


In [13]:
all_test_small = all_test[10:15]

In [14]:
all_test_small['stars'] = all_test_small['text'].apply(lambda x: sentiment_analysis_bert_base_multilingual_uncased(x))

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Sentiment Synthetic Score Function

In [19]:
def stars_mean_to_score(review):
    
    points = []

    for classifications in review:
        for classification in classifications:

            #Mean of fragment scores x2 as Spanish score range is 1-10
            grade = int(classification['label'].split(' ')[0])
            points.append(grade)  
        score = round(np.mean(points), 2) * 2

    return(score)   

In [20]:
all_test_small['stars_mean'] = all_test_small['stars'].apply(lambda x: stars_mean_to_score(x))

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
all_test_small

,site,url_link,author,game,company,genre,platform,text,score,stars,stars_mean
10,Vandal,https://vandal.elespanol.com/analisis/ps4/mega...,Carlos Leiva,Mega Man X Legacy Collection - Análisis,Capcom,Plataformas 2D Run and gun,PS4 PC Xbox One Switch,Pocas cosas hay que Capcom esté cuidando más a...,8.8,"[[{'label': '3 stars', 'score': 0.493329465389...",7.76
11,meristation,https://as.com/meristation/2016/03/18/analisis...,None,We Are The Dwarves,None,"Acción, Aventura, Tactical",PC,"Ese es el caso de We are the Dwarves!, un jueg...",7.0,"[[{'label': '3 stars', 'score': 0.470475465059...",6.28
12,meristation,https://as.com/meristation/2008/09/01/analisis...,Joaquín Relaño,Ratchet & Clank: En busca del tesoro,None,"Acción, Aventura",PS3,Es curioso comprobar cómo los profundos cambio...,7.0,"[[{'label': '4 stars', 'score': 0.309142857789...",7.50
13,3D Juegos,https://www.3djuegos.com/juegos/analisis/12815...,Alberto Pastor,Diablo III - Análisis,None,None,PC PS3 X360 Mac,"Referente en PC desde hace casi dos décadas, l...",8.8,"[[{'label': '4 stars', 'score': 0.555725336074...",8.00
14,Vandal,https://vandal.elespanol.com/analisis/pc/tyran...,Carlos Leiva,Tyranny - Análisis,Paradox Interactive,RPG Occidental Rol,PC,¿Cuántas veces habremos salvado ya al mundo de...,8.0,"[[{'label': '4 stars', 'score': 0.402889281511...",7.20


### Precission Score

In [31]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall

SyntaxError: unexpected EOF while parsing (<ipython-input-31-b27eff294fca>, line 12)

## 3 Fine Tuning BERT model classifier

3.1 Train the entire architecture 
3.2 Train some layers while freezing others
3.3 Freeze the entire architecture 

https://huggingface.co/transformers/training.html

In [27]:
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment', return_dict=True)
model.train()

"model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment', return_dict=True)\nmodel.train()"

In [25]:
from transformers import AdamW 

optimizer = AdamW(model.parameters(), lr=1e-5)

In [26]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)

In [29]:
from transformers import BertTokenizer
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
text_batch = ["I love Pixar.", "I don't care for Pixar."]
encoding = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

In [32]:
from sklearn.model_selection import train_test_split

In [41]:
train_dataset, test_dataset = train_test_split(all_test_small['text'], random_state=2018, test_size=0.2)

In [43]:
test_dataset

10    Pocas cosas hay que Capcom esté cuidando más a...
Name: text, dtype: object

In [44]:
from transformers import Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

In [45]:
trainer.train()
#trainer.evaluate()

KeyError: 2

In [36]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }